In [10]:
# 📥 Importações
%pip install pandas
import librosa
import librosa.feature
import numpy as np
import pandas as pd

# 📂 Carrega o áudio
audio_path = '../audio/15-11 - 1_00000_000.wav'
y, sr = librosa.load(audio_path, sr=None)

# 🪟 Parâmetros da janela
win_sec = 5
win_len = win_sec * sr

# ⏱️ Intervalos com bugios (cada um com 10s de duração)
bugio_intervals = [
    (28*60 + 17, 28*60 + 24),
    (29*60 + 40, 29*60 + 50),
    (30*60 +  0, 30*60 + 10),
    (30*60 + 30, 30*60 + 40),
    (30*60 + 40, 30*60 + 50),
    (31*60 + 10, 31*60 + 20),
    (32*60 +  0, 32*60 + 10),
    (32*60 + 30, 32*60 + 40),
    (34*60 + 20, 34*60 + 30),
    (36*60 + 39, 36*60 + 49),
    (37*60 +  0, 37*60 + 10),
    (38*60 + 20, 38*60 + 30),
    (38*60 + 30, 38*60 + 40),
    (42*60 +  0, 42*60 + 10),
    (42*60 + 20, 42*60 + 30),
    (43*60 +  9, 43*60 + 19),
    (43*60 + 20, 43*60 + 30)
]

# 📊 Armazenar features
features = []

# 🎛️ Processa cada janela de 5 segundos
for i in range(0, len(y), win_len):
    y_seg = y[i:i+win_len]
    if len(y_seg) < win_len:
        break
    
    start_time = i / sr
    end_time = (i + win_len) / sr

    # 🎯 Rótulo: 1 se a janela tiver pelo menos 1s de interseção com qualquer intervalo de bugios
    label = 0
    for start_b, end_b in bugio_intervals:
        intersection = min(end_time, end_b) - max(start_time, start_b)
        if intersection >= 1.0:
            label = 1
            break

    # 📈 Features
    energy = np.sum(y_seg**2)
    zcr = librosa.feature.zero_crossing_rate(y_seg)[0].mean()
    centroid = librosa.feature.spectral_centroid(y=y_seg, sr=sr)[0].mean()
    bandwidth = librosa.feature.spectral_bandwidth(y=y_seg, sr=sr)[0].mean()
    flatness = librosa.feature.spectral_flatness(y=y_seg)[0].mean()
    mfccs = librosa.feature.mfcc(y=y_seg, sr=sr, n_mfcc=13)
    mfccs_mean = mfccs.mean(axis=1)

    row = {
        "start_time": start_time,
        "end_time": end_time,
        "label": label,
        "energy": energy,
        "zcr": zcr,
        "centroid": centroid,
        "bandwidth": bandwidth,
        "flatness": flatness
    }

    for j, val in enumerate(mfccs_mean):
        row[f'mfcc_{j+1}'] = val

    features.append(row)

# 🧾 DataFrame final com tempos formatados
df = pd.DataFrame(features)
df['start_min'] = df['start_time'].apply(lambda x: f"{int(x//60)}:{int(x%60):02}")
df['end_min'] = df['end_time'].apply(lambda x: f"{int(x//60)}:{int(x%60):02}")

# ✅ Visualização de exemplo
df.head()


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


,start_time,end_time,label,energy,zcr,centroid,bandwidth,flatness,mfcc_1,mfcc_2,...,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,start_min,end_min
0,0.0,5.0,0,363.300415,0.008789,2390.613876,2810.942312,0.001530,-298.275146,48.550407,...,25.400360,6.961646,20.940340,16.245693,-0.196101,5.898236,19.263458,-9.531937,0:00,0:05
1,5.0,10.0,0,365.222961,0.008737,2750.017121,2949.635575,0.003642,-286.879547,35.441349,...,22.555412,8.646086,21.021854,15.721250,0.193035,4.019448,19.427181,-4.460641,0:05,0:10
2,10.0,15.0,0,373.430603,0.018002,3420.207958,3184.696882,0.008090,-258.784058,16.643496,...,19.066120,11.145739,20.242632,14.219850,-0.105031,2.136864,15.678128,1.886264,0:10,0:15
3,15.0,20.0,0,369.935822,0.014266,3135.896961,3127.100403,0.004975,-277.900818,20.349672,...,20.768684,10.876724,21.483610,16.573643,-0.864002,3.025763,15.478945,-0.565292,0:15,0:20
4,20.0,25.0,0,368.645477,0.009737,2354.352503,2750.280172,0.001448,-302.599670,52.482407,...,24.259644,7.997576,21.548424,21.959330,-3.353864,6.374354,18.244440,-6.488849,0:20,0:25


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# ⚙️ Dados
X = df.drop(columns=['start_time', 'end_time', 'start_min', 'end_min', 'label'])
y = df['label']

# 📊 Verifique a distribuição das classes
print("Distribuição das classes em y:")
print(y.value_counts())

# 🔀 Divisão treino/teste (com estratificação)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

print("\nDistribuição das classes em y_train:")
print(y_train.value_counts())
print("\nDistribuição das classes em y_test:")
print(y_test.value_counts())

# 🎯 Modelo de Regressão Logística
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 📊 Avaliação do modelo
y_pred = model.predict(X_test)
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

Distribuição das classes em y:
label
0    683
1     36
Name: count, dtype: int64

Distribuição das classes em y_train:
label
0    512
1     27
Name: count, dtype: int64

Distribuição das classes em y_test:
label
0    171
1      9
Name: count, dtype: int64

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       171
           1       0.80      0.44      0.57         9

    accuracy                           0.97       180
   macro avg       0.89      0.72      0.78       180
weighted avg       0.96      0.97      0.96       180

Matriz de Confusão:
[[170   1]
 [  5   4]]


/home/leonardo/Projects/bugio-audio-analysis/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
print(df['label'].value_counts())  # Agora deve mostrar pelo menos alguns valores 1


label
0    683
1     36
Name: count, dtype: int64
